In [1]:
import pandas as pd
import numpy as np
# import cPickle 
import _pickle as cPickle
from nltk.corpus import stopwords
from gensim.models import word2vec
import nltk.data
import re
import logging
from nltk.stem.snowball import *
import itertools

# Python 2.x:
# import HTMLParser
# html_parser = HTMLParser.HTMLParser()
import html.parser as html_parser

import multiprocessing
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

In [2]:
TEST_CASE = {}

In [4]:
import re
# import HTMLParser
# html_parser = HTMLParser.HTMLParser()

import html.parser as html_parser

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

nltk.download("wordnet", "/Users/bharath/sourcecode/cynor/skill2vec/nltk_data/")
nltk.data.path.append('./nltk_data/')


import sys
if sys.version_info[0] >= 3:
    unicode = str

def skill_transform(skill, remove_stopwords = True):
    skill = unicode(skill)
    skill = html_parser.unescape(skill)
    
    skill = skill.replace("_", " ").split()
    skill = " ".join([sk for sk in skill if sk])
    
    skill = re.sub(r"\(.*\)", "", skill)
    skill = skill.replace("-", "") \
        .replace(".", "") \
        .replace(",", "") \
        .replace("-", "") \
        .replace(":", "") \
        .replace("(", "") \
        .replace(")", "") \
        .replace(u"åá", "") \
        .replace(u"&", "and") \
        .replace(" js", "js") \
        .replace("-js", "js") \
        .replace("_js", "js") \
        .replace("java script", "js") 
    
    skill = skill.lower()
    
    # Special cases replace
    special_case = {}
    special_case["javascript"] = [ "js", "java script", "javascripts", "java scrip" ]
    special_case["wireframe"] = [ "wireframes", "wire frame", "wire frames", "wire-frame", "wirefram", "wire fram", "wireframing" ]
    special_case["OOP"] = [  "object oriented", "object oriented programming", ]
    special_case["OOD"] = [ "object oriented design", ]
    special_case["OLAP"] = [ "online analytical processing",  ]
    special_case["Ecommerce"] = [ "e commerce",  ]
    special_case["consultant"] = [ "consulting",  ]
    special_case["ux"] = [ "user experience", "web user experience design", "user experience design", "ux designer", "user experience/ux" ]
    special_case["html5"] = [ "html 5",  ]
    special_case["j2ee"] = [ "jee",  ]
    special_case["osx"] = [ "mac os x", "os x" ]
    special_case["senior"] = [ "sr" ]
    special_case["qa"] = [ "quality",  ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["webservice"] = [ "webservices", "website", "webapps" ]
    special_case["xml"] = [ "xml file", "xml schemas", "xml/json", "xml web service" ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["nlp"] = [ "natural language process", "natural language", "nltk" ]
    special_case["aws"] = [ "amazon web service", ]
    
    for root_skill in special_case:
        if skill in special_case[root_skill]:
            skill = root_skill
    
    # Special case regex
    special_case_regex = {
        r'^angular.*$': 'angularjs',
        r'^node.*$': 'nodejs',
        r'^(.*)[_\s]js$': '\\1js',
        r'^(.*) js$': '\\1js',
        r'^(.*) (and|or).*$': '\\1',
    }
    for regex_rule in special_case_regex:
        after_skill = re.sub(regex_rule, special_case_regex[regex_rule], skill)
        if after_skill != skill:
            skill = after_skill
            break
    
    # Stem
    if len(skill) > 2:
        skill_after = skill.split(" ")
        skill_after = [wordnet_lemmatizer.lemmatize(sk, pos="v") for sk in skill_after]
        skill_after = " ".join(skill_after)
        skill = skill_after
    
    # skill stopwords 
    if remove_stopwords:
        skill_stopwords = [ "app", "touch", "the", "application", "programming", "program", "design"
                           "developer", "framework", "development", "programmer", "technologies",
                          "advance", "core"]
        skill_after = skill.split(" ")
        skill = " ".join([ sk for sk in skill_after if sk not in skill_stopwords ])
    
    skill = skill.lower().strip().replace(" ", "_")
    skill = re.sub(' +',' ', skill)
    
    # NOTE: replace js tail
    skill = re.sub('js$','', skill)
    
    return skill

skill_transform("js")
skill_transform("angularjs")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bharath/sourcecode/cynor/skill2vec/nltk_data/..
[nltk_data]     .
[nltk_data]   Package wordnet is already up-to-date!


'angular'

In [5]:
skill_transform("html5_and_css3_or_html6")

'html5_and_css3'

In [6]:
def test_case(model, model_name):
    cases = [ "machine learning", "js", "javascript", "python", "html", "html5", "css", "angular", "nodejs" ]
    TEST_CASE[model_name] = []
    for case in cases:
        try:
            TEST_CASE[model_name].append(model.similar_by_word(skill_transform(case)))
        except:
            pass

In [8]:
# data_must_have = pd.read_csv('mustHaveSkills-2.csv', header = 0, encoding='ISO-8859-1')
data_must_have = pd.read_csv('formatted_skill2vec.csv', header = 0, encoding='ISO-8859-1', low_memory=False)
data_must_have.drop_duplicates(inplace=True, keep='last')
data_must_have = data_must_have[data_must_have["job_title"] != 0]
del data_must_have["job_brief_id"]
# data_must_have['Count'] = data_must_have.groupby('job_title')['keyword_name'].transform(pd.Series.value_counts)
data_must_have.drop_duplicates(inplace=True)

unique_skill_before = len(data_must_have['keyword_name'].unique())
print("Before: %s" % unique_skill_before)

data_must_have['skill'] = data_must_have['keyword_name'].apply(skill_transform)

unique_skill_after = len(data_must_have['skill'].unique())
print("After: %s" % unique_skill_after)

print("Resize: %s%%" % (unique_skill_after * 100.0 / unique_skill_before))

# data_must_have['keyword_name'] = data_must_have['keyword_name'].str.replace(' ' ,'_')
# data_must_have

Before: 754
After: 697
Resize: 92.44031830238727%


In [9]:
# pd.set_option('display.max_rows', 1500)
# data_must_have.drop_duplicates(subset=["keyword_name", "skill"])[["keyword_name", "skill"]].sort_values("skill")[3000:]
musthave_data = data_must_have.groupby('job_title')['skill'].apply(list).tolist()

print ("Training model...")
musthave_model = word2vec.Word2Vec(musthave_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_model.init_sims(replace=True)
test_case(musthave_model, "musthave_model")

2021-10-01 16:19:08,277 : INFO : collecting all words and their counts
2021-10-01 16:19:08,278 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 16:19:08,284 : INFO : collected 697 word types from a corpus of 8252 raw words and 5492 sentences
2021-10-01 16:19:08,291 : INFO : Creating a fresh vocabulary
2021-10-01 16:19:08,333 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 697 unique words (100.0%% of original 697, drops 0)', 'datetime': '2021-10-01T16:19:08.298355', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-10-01 16:19:08,339 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 8252 word corpus (100.0%% of original 8252, drops 0)', 'datetime': '2021-10-01T16:19:08.338924', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.1

Training model...


2021-10-01 16:19:08,476 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-10-01 16:19:08,486 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-10-01 16:19:08,488 : INFO : EPOCH - 2 : training on 8252 raw words (7071 effective words) took 0.0s, 250966 effective words/s
2021-10-01 16:19:08,514 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-10-01 16:19:08,514 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-10-01 16:19:08,516 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-10-01 16:19:08,539 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-10-01 16:19:08,540 : INFO : EPOCH - 3 : training on 8252 raw words (7085 effective words) took 0.0s, 179458 effective words/s
2021-10-01 16:19:08,561 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-10-01 16:19:08,564 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-10-01

In [11]:
with open("prep_data_tokens_underscore_1", "rb") as g:
    data_dice = cPickle.load(g)
dice_data = [ [ skill_transform(d.replace("_", " ")) for d in dice if d is not None ] for dice in data_dice if dice is not None ]


musthave_dice_data = dice_data + musthave_data
musthave_dice_data = [ list(set([ d for d in data if d is not None and len(d) > 0 ])) for data in musthave_dice_data  ]

print(len(musthave_dice_data))
musthave_dice_data

print ("Training model...")
musthave_dice_model = word2vec.Word2Vec(musthave_dice_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_dice_model.init_sims(replace=True)
test_case(musthave_dice_model, "musthave_dice_model")

2021-10-01 16:20:13,622 : INFO : collecting all words and their counts
2021-10-01 16:20:13,623 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 16:20:13,698 : INFO : PROGRESS: at sentence #10000, processed 187861 words, keeping 8491 word types
2021-10-01 16:20:13,710 : INFO : collected 8509 word types from a corpus of 191437 raw words and 12638 sentences
2021-10-01 16:20:13,714 : INFO : Creating a fresh vocabulary
2021-10-01 16:20:13,801 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 8509 unique words (100.0%% of original 8509, drops 0)', 'datetime': '2021-10-01T16:20:13.801840', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-10-01 16:20:13,803 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 191437 word corpus (100.0%% of original 191437, drops 0)', 'datetime': '2021-10-01T16:20:13.8

12638
Training model...


2021-10-01 16:20:13,955 : INFO : deleting the raw counts dictionary of 8509 items
2021-10-01 16:20:13,956 : INFO : sample=0.001 downsamples 0 most-common words
2021-10-01 16:20:13,959 : INFO : Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 191437 word corpus (100.0%% of prior 191437)', 'datetime': '2021-10-01T16:20:13.959572', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-10-01 16:20:14,102 : INFO : estimated required memory for 8509 words and 100 dimensions: 11061700 bytes
2021-10-01 16:20:14,103 : INFO : resetting layer weights
2021-10-01 16:20:14,112 : INFO : Word2Vec lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2021-10-01T16:20:14.112886', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'build_vocab'}
2021-10-01 16:20:14,113 : INF

In [65]:
# model.similar_by_word(skill_transform('machine learning'))

In [66]:
# model.similar_by_word('python')

In [13]:
musthave_dice_naruki_data = musthave_dice_data
data_naruki = pd.read_csv('naukri_skill_full', header = 0, encoding='ISO-8859-1')
data_naruki.drop_duplicates(subset=['id', 'skill'], keep='last')

data_naruki['skill'] = data_naruki['skill'].apply(skill_transform)
data_naruki_final = data_naruki.groupby('id')['skill'].apply(list)
print(len(data_naruki_final))

for skills in data_naruki_final:
    if len(skills) > 2 and skills not in musthave_dice_naruki_data:
        musthave_dice_naruki_data.append(skills)
        
len(musthave_dice_naruki_data)

120107


110504

In [15]:
# import cPickle as pickle
import _pickle as pickle

pickle.dump( musthave_dice_naruki_data, open( "cynor_model.p", "wb" ) )

In [16]:
print ("Training model...")
musthave_dice_naruki_model = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=3, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_dice_naruki_model.init_sims(replace=True)
test_case(musthave_dice_naruki_model, "musthave_dice_naruki_model")

2021-10-01 16:25:37,394 : INFO : collecting all words and their counts
2021-10-01 16:25:37,396 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 16:25:37,449 : INFO : PROGRESS: at sentence #10000, processed 187861 words, keeping 8491 word types
2021-10-01 16:25:37,473 : INFO : PROGRESS: at sentence #20000, processed 254725 words, keeping 18159 word types
2021-10-01 16:25:37,538 : INFO : PROGRESS: at sentence #30000, processed 341184 words, keeping 27356 word types
2021-10-01 16:25:37,577 : INFO : PROGRESS: at sentence #40000, processed 427836 words, keeping 35384 word types
2021-10-01 16:25:37,613 : INFO : PROGRESS: at sentence #50000, processed 518155 words, keeping 43089 word types


Training model...


2021-10-01 16:25:37,655 : INFO : PROGRESS: at sentence #60000, processed 604765 words, keeping 49209 word types
2021-10-01 16:25:37,703 : INFO : PROGRESS: at sentence #70000, processed 689611 words, keeping 56180 word types
2021-10-01 16:25:37,749 : INFO : PROGRESS: at sentence #80000, processed 774503 words, keeping 62461 word types
2021-10-01 16:25:37,788 : INFO : PROGRESS: at sentence #90000, processed 861508 words, keeping 67633 word types
2021-10-01 16:25:37,829 : INFO : PROGRESS: at sentence #100000, processed 951473 words, keeping 72327 word types
2021-10-01 16:25:37,873 : INFO : PROGRESS: at sentence #110000, processed 1037678 words, keeping 77970 word types
2021-10-01 16:25:37,878 : INFO : collected 78061 word types from a corpus of 1042225 raw words and 110504 sentences
2021-10-01 16:25:37,879 : INFO : Creating a fresh vocabulary
2021-10-01 16:25:38,019 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 21185 unique words (27.139032295256275%% of origina

2021-10-01 16:26:05,201 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=21185, vector_size=100, alpha=0.025)', 'datetime': '2021-10-01T16:26:05.201897', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
/tmp/ipykernel_10602/1428038617.py:12: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  musthave_dice_naruki_model.init_sims(replace=True)
2021-10-01 16:26:05,212 : WARNING : destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


In [17]:
TEST_CASE

{'musthave_model': [],
 'musthave_dice_model': [],
 'musthave_dice_naruki_model': []}

In [18]:
TEST_CASE_TABLE = [ pd.DataFrame(TEST_CASE[t]) for t in TEST_CASE ]

In [19]:
[ t for t in TEST_CASE ]

['musthave_model', 'musthave_dice_model', 'musthave_dice_naruki_model']

In [20]:
TEST_CASE_TABLE[0]

""


In [21]:
TEST_CASE_TABLE[1]

""


In [22]:
TEST_CASE_TABLE[2]

""


In [23]:
musthave_dice_naruki_model.wv.save_word2vec_format("musthave_dice_naruki.model")

2021-10-01 16:27:43,944 : INFO : storing 21185x100 projection weights into musthave_dice_naruki.model


In [24]:
!ls -l

total 246456
-rw-r--r--@  1 bharath  staff      6148 Oct  1 16:13 .DS_Store
drwxr-xr-x  13 bharath  staff       416 Oct  1 15:35 .git
-rw-r--r--   1 bharath  staff      1799 Oct  1 15:34 .gitignore
drwxr-xr-x   4 root     staff       128 Oct  1 16:17 .ipynb_checkpoints
-rw-r--r--   1 bharath  staff        11 Oct  1 15:34 README.md
-rw-r--r--   1 root     staff  14381180 Oct  1 16:25 cynor_model.p
-rw-r--r--   1 bharath  staff     67063 Oct  1 16:17 format_data.ipynb
-rw-r--r--   1 root     staff    252385 Oct  1 16:13 formatted_skill2vec.csv
-rw-r--r--@  1 bharath  staff   8885546 Sep 30 21:56 mustHaveSkills-2.csv
-rw-r--r--   1 root     staff  25538370 Oct  1 16:27 musthave_dice_naruki.model
-rw-r--r--   1 bharath  staff  18240737 Sep 30 21:56 naukri_skill_full
drwxr-xr-x   4 bharath  staff       128 Oct  1 15:35 nltk_data
-rw-r--r--   1 bharath  staff   3972850 Sep 30 21:56 prep_data_tokens_underscore_1
-rw-r--r--@  1 bharath  staff   1074156 Oct  1 14:39 skill2vec_1K.csv
-rw-r--r--@

In [55]:
cynor_skill2vec_model_100d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=100, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

cynor_skill2vec_model_200d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=200, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

2021-10-01 16:51:31,157 : INFO : collecting all words and their counts
2021-10-01 16:51:31,159 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 16:51:31,218 : INFO : PROGRESS: at sentence #10000, processed 187861 words, keeping 8491 word types
2021-10-01 16:51:31,244 : INFO : PROGRESS: at sentence #20000, processed 254725 words, keeping 18159 word types
2021-10-01 16:51:31,294 : INFO : PROGRESS: at sentence #30000, processed 341184 words, keeping 27356 word types
2021-10-01 16:51:31,365 : INFO : PROGRESS: at sentence #40000, processed 427836 words, keeping 35384 word types
2021-10-01 16:51:31,395 : INFO : PROGRESS: at sentence #50000, processed 518155 words, keeping 43089 word types
2021-10-01 16:51:31,437 : INFO : PROGRESS: at sentence #60000, processed 604765 words, keeping 49209 word types
2021-10-01 16:51:31,476 : INFO : PROGRESS: at sentence #70000, processed 689611 words, keeping 56180 word types
2021-10-01 16:51:31,518 : INFO : PROGRESS: at s

2021-10-01 16:51:52,088 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=15534, vector_size=100, alpha=0.025)', 'datetime': '2021-10-01T16:51:52.088691', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2021-10-01 16:51:52,090 : INFO : collecting all words and their counts
2021-10-01 16:51:52,092 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 16:51:52,161 : INFO : PROGRESS: at sentence #10000, processed 187861 words, keeping 8491 word types
2021-10-01 16:51:52,188 : INFO : PROGRESS: at sentence #20000, processed 254725 words, keeping 18159 word types
2021-10-01 16:51:52,223 : INFO : PROGRESS: at sentence #30000, processed 341184 words, keeping 27356 word types
2021-10-01 16:51:52,258 : INFO : PROGRESS: at sentence #40000, processed 427836 words, keeping 35384 word types
2021-10-01 16:51:52,296 : INFO : PROGRESS: at sentence #50000,

2021-10-01 16:52:12,091 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-10-01 16:52:12,092 : INFO : EPOCH - 5 : training on 1042225 raw words (932332 effective words) took 4.2s, 220259 effective words/s
2021-10-01 16:52:12,093 : INFO : Word2Vec lifecycle event {'msg': 'training on 5211125 raw words (4662323 effective words) took 18.8s, 247740 effective words/s', 'datetime': '2021-10-01T16:52:12.093074', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'train'}
2021-10-01 16:52:12,094 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=15534, vector_size=100, alpha=0.025)', 'datetime': '2021-10-01T16:52:12.094365', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [56]:
cynor_skill2vec_model_100d.wv.save_word2vec_format("cynor_skill2vec_100d.model")
cynor_skill2vec_model_200d.wv.save_word2vec_format("cynor_skill2vec_200d.model")

2021-10-01 16:52:12,124 : INFO : storing 15534x100 projection weights into cynor_skill2vec_100d.model
2021-10-01 16:52:13,622 : INFO : storing 15534x100 projection weights into cynor_skill2vec_200d.model


In [57]:
!ls -l
!free -m

total 317144
-rw-r--r--@  1 bharath  staff      6148 Oct  1 16:13 .DS_Store
drwxr-xr-x  13 bharath  staff       416 Oct  1 15:35 .git
-rw-r--r--   1 bharath  staff      1799 Oct  1 15:34 .gitignore
drwxr-xr-x   4 root     staff       128 Oct  1 16:17 .ipynb_checkpoints
-rw-r--r--   1 bharath  staff        11 Oct  1 15:34 README.md
-rw-r--r--   1 root     staff  14381180 Oct  1 16:25 cynor_model.p
-rw-r--r--   1 root     staff  18111038 Oct  1 16:52 cynor_skill2vec_100d.model
-rw-r--r--   1 root     staff  18104991 Oct  1 16:52 cynor_skill2vec_200d.model
-rw-r--r--   1 bharath  staff     67064 Oct  1 16:49 format_data.ipynb
-rw-r--r--   1 root     staff    252385 Oct  1 16:13 formatted_skill2vec.csv
-rw-r--r--@  1 bharath  staff   8885546 Sep 30 21:56 mustHaveSkills-2.csv
-rw-r--r--   1 root     staff  25538370 Oct  1 16:27 musthave_dice_naruki.model
-rw-r--r--   1 bharath  staff  18240737 Sep 30 21:56 naukri_skill_full
drwxr-xr-x   4 bharath  staff       128 Oct  1 15:35 nltk_data
-rw-

In [58]:
musthave_dice_naruki_model.wv.similar_by_word(skill_transform('javascript'),topn=10)

[('html5', 0.8158990144729614),
 ('css3', 0.813828706741333),
 ('ajax', 0.8093597888946533),
 ('oops_concept', 0.8039079904556274),
 ('webdeveloper', 0.7994744777679443),
 ('jquery', 0.7985265851020813),
 ('web_tool', 0.7981303930282593),
 ('bootstrap', 0.7941082715988159),
 ('photoshop_developer', 0.7925956845283508),
 ('web_developement', 0.7914119958877563)]

In [72]:
cynor_skill2vec_model_200d.wv.similar_by_word(skill_transform('gcp'))

[('ardl', 0.9357483386993408),
 ('cell_culture', 0.9179457426071167),
 ('downstream_process', 0.9159078001976013),
 ('psur', 0.9125140905380249),
 ('xrd', 0.9094202518463135),
 ('drug_regulatory_affairs', 0.90939861536026),
 ('ard', 0.9073624014854431),
 ('glp', 0.904293417930603),
 ('dq', 0.9041879773139954),
 ('regulatory_market', 0.9041591286659241)]

In [32]:
# [(u'css3', 0.9307963252067566),
#  (u'bootstrap', 0.9234713912010193),
#  (u'ui_developer', 0.9192823171615601),
#  (u'front_end_engineer', 0.9077576398849487),
#  (u'front__end', 0.9058327674865723),
#  (u'css_3', 0.9039251208305359),
#  (u'ui_engineer', 0.9014566540718079),
#  (u'adobe_experience_management_suite', 0.9006956815719604),
#  (u'html4', 0.9003456234931946),
#  (u'user_interface_developer', 0.8981920480728149)]

In [29]:
musthave_dice_naruki_model.similar_by_word(skill_transform('javascript'))

AttributeError: 'Word2Vec' object has no attribute 'similar_by_word'

In [84]:
musthave_dice_naruki_model.similar_by_word(skill_transform('node_js'), 300)

[(u'react', 0.9388986825942993),
 (u'backbone', 0.9247055053710938),
 (u'mean_stack', 0.9243057370185852),
 (u'angular', 0.9219822883605957),
 (u'mean', 0.9090983867645264),
 (u'bower', 0.9059317111968994),
 (u'full_stack_developer', 0.9024332165718079),
 (u'npm', 0.9016261100769043),
 (u'frontend', 0.9010946750640869),
 (u'grunt', 0.8981549143791199),
 (u'css/javascript', 0.8971441984176636),
 (u'gulp', 0.8954569697380066),
 (u'bootstrap', 0.8946527242660522),
 (u'mongodb', 0.894255518913269),
 (u'javascript_developer', 0.8926291465759277),
 (u'mean_stack_developer', 0.8926271200180054),
 (u'mongo_db', 0.8904428482055664),
 (u'front_end_web_developer', 0.8894308805465698),
 (u'service_workers', 0.8887876272201538),
 (u'ui_architect', 0.8884862661361694),
 (u'less', 0.888136088848114),
 (u'loopback', 0.8872900009155273),
 (u'firebase', 0.8807092905044556),
 (u'mocha', 0.8802285194396973),
 (u'meanstack', 0.880145788192749),
 (u'front__end', 0.8791353106498718),
 (u'javascript_mvc', 0.8

In [85]:
musthave_dice_naruki_model_100d.similar_by_word(skill_transform('node_js'), 300)

[(u'react', 0.9410043358802795),
 (u'backbone', 0.9235603213310242),
 (u'angular', 0.9139069318771362),
 (u'mean_stack', 0.9107990860939026),
 (u'npm', 0.8981935977935791),
 (u'mean', 0.8961061239242554),
 (u'mongodb', 0.8947668075561523),
 (u'bower', 0.89299476146698),
 (u'gulp', 0.8898729085922241),
 (u'full_stack_developer', 0.8898143768310547),
 (u'less', 0.8879895806312561),
 (u'grunt', 0.8874234557151794),
 (u'bootstrap', 0.8813268542289734),
 (u'javascript_mvc', 0.8796409368515015),
 (u'ui_architect', 0.878745436668396),
 (u'express', 0.8783759474754333),
 (u'javascript_developer', 0.8779905438423157),
 (u'loopback', 0.8772644996643066),
 (u'scss', 0.876909077167511),
 (u'meanstack', 0.8767061233520508),
 (u'fullstack_developer', 0.8745248317718506),
 (u'frontend_developer', 0.8743304014205933),
 (u'oo_javascript', 0.8734292984008789),
 (u'frontend', 0.8726052641868591),
 (u'mean_stack_developer', 0.8722931146621704),
 (u'mocha', 0.8718781471252441),
 (u'ionic', 0.86925601959228

In [86]:
re.sub(r'^angular.*$', 'angular', "angular.js")

'angular'